In [1]:
import random
import numpy as np
import csv
with open("wine-dataset.csv") as f:
    next(f, None)
    Tdata=[]
    for line in csv.reader(f,delimiter=","):
        row=[float(x) for x in line]
        Tdata.append(row)

# entropy function
def entropy(data):
    
    lastIndex=len(data[0])-1
    n0,n1=0,0
    for i in data:
        if i[lastIndex]==0:
            n0=n0+1
        else:
            n1=n1+1
    p1=n0/(n1+n0)
    p2=1-p1
    if p1 != 0:
        p1=-p1*np.log(p1)
    if p2 != 0:
        p2=-p2*np.log(p2)
    return p1+p2


# The Best split function
def BestSplit(Data):
    RowSize=len(Data[0])
    if RowSize<=1:
        exit
    infoGain=1e-9
    Left=[]
    Right=[]
    Threshold=None
    Feature=None
    
        
    for feature in range(RowSize-1):
        allValsinFeature=[Data[x][feature] for x in range(len(Data))]
        minVal=min(allValsinFeature)
        maxVal=max(allValsinFeature)
        Tvals = np.linspace(minVal,maxVal,6,endpoint=False)[1:]
        for threshold in Tvals:
            left=[]
            right=[]
            for inputs in Data:
                if inputs[feature]<threshold:
                    left.append([inputs[x] for x in [*range(0,feature),*range(feature+1,RowSize)]])
                else:
                    right.append([inputs[x] for x in [*range(0,feature),*range(feature+1,RowSize)]])
            # print("threshold:",threshold,"left:",left,"right:",right)
            if(len(left)>0 and len(right)>0):
                delEntropy=entropy(Data)-len(left)/len(Data)*entropy(left)-len(right)/len(Data)*entropy(right)
                if delEntropy > infoGain:
                    infoGain=delEntropy
                    Left=left
                    Right=right
                    Threshold=threshold
                    Feature=feature
    return {"threshold":Threshold,"feature":Feature,"Left":Left,"Right":Right}


class DecisionTree():
    tree={}

    def learn(self,Data):
        tree={'feature':None, 'Value':None ,'threshold':None,'leftTree':{},'rightTree':{}}
        if len(Data)>0:
            # there should be at least 1 feature to split on and all the data shouldn't go to a single node
            if len(Data[0])>=2 and entropy(Data)!=0:
                best_split=BestSplit(Data)
                tree['feature']=best_split["feature"]
                tree['threshold']=best_split["threshold"]
                # print(best_split)
                leftTree={}
                rightTree={}
                leftTree=DecisionTree.learn(self,best_split["Left"])
                rightTree=DecisionTree.learn(self,best_split["Right"])
                tree['leftTree']=leftTree
                tree['rightTree']=rightTree
                # if not tree['feature'] and not tree['Value']:
                    # tree['Value']=0
                return tree 
            else :
                tree['leftTree']= None
                tree['rightTree']= None 
                rowsize=len(Data[0])
                labels=[Data[x][rowsize-1] for x in range(len(Data))]
                tree['Value']=max(labels,key=labels.count)
                return tree    

    def classify(self,tree,test_instance):
        if tree:
            if(tree['Value']) is not None:
                return tree['Value'] 
            else:
                f = test_instance[tree['feature']]
                t =[test_instance[x] for x in [*range(0,tree['feature']),*range(tree['feature']+1,len(test_instance))]] 
                if(f<tree['threshold']):
                    return self.classify(tree['leftTree'],t)
                else:
                    return self.classify(tree['rightTree'],t)if len(Data)>0:
    

def run_decision_tree():
    BestTree={}
    BestAccuracy=0
    avgAccuracy=0

    # Split training/test sets
    K = 10
    for kfold in range(K):
        training_set = [x for i, x in enumerate(Tdata) if i % K != kfold]
        test_set = [x for i, x in enumerate(Tdata) if i % K == kfold]
        # print(len(training_set[0]))
        Tree = DecisionTree()
        # Construct a tree using training set
        tree=Tree.learn( training_set )
        # print(tree)
        
        # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = Tree.classify(tree,instance[:-1] )
            results.append( result == instance[-1])

        # Accuracy
        accuracy = float(results.count(True))/float(len(results))
        if BestAccuracy<accuracy:
            BestAccuracy=accuracy
            BestTree=tree
        avgAccuracy=avgAccuracy+accuracy
        print("kfold: %d, accuracy: %.4f" % (kfold+1, accuracy))
        

        # Writing results to a file (DO NOT CHANGE)
        f = open("Chirag-Mehta-"+"result.txt", "w")
        f.write("accuracy: %.4f" % accuracy)
        f.close()
    avgAccuracy=avgAccuracy/K
    print("Best accuracy: %f, average accuracy: %f" %(BestAccuracy,avgAccuracy))
    f = open("Chirag-Mehta-"+"result.txt", "w")
    f.write("average accuracy: %.4f" % avgAccuracy)
    f.close()
if __name__ == "__main__":
    run_decision_tree()

kfold: 1, accuracy: 0.8122
kfold: 2, accuracy: 0.8388
kfold: 3, accuracy: 0.8020
kfold: 4, accuracy: 0.8000
kfold: 5, accuracy: 0.8163
kfold: 6, accuracy: 0.8000
kfold: 7, accuracy: 0.8347
kfold: 8, accuracy: 0.7918
kfold: 9, accuracy: 0.7935
kfold: 10, accuracy: 0.8037
Best accuracy: 0.838776, average accuracy: 0.809306


In [14]:
# improvement Strategy used: Gini Index

import random
import numpy as np
import csv

# Storing the test and training data in TData
with open("wine-dataset.csv") as f:
    next(f, None)
    Tdata=[]
    for line in csv.reader(f,delimiter=","):
        row=[float(x) for x in line]
        Tdata.append(row)

# gini Index function
def gini(data):
    
    lastIndex=len(data[0])-1
    n0,n1=0,0
    for i in data:
        if i[lastIndex]==0:
            n0=n0+1
        else:
            n1=n1+1
    p1=n0/(n1+n0)
    p2=1-p1
    # 1- sum p_i^2
    return 1-p1**2-p2**2


# The Best split function

def BestSplit(Data):
    RowSize=len(Data[0])
    if RowSize<=1:
        exit
    # initializing the variables
    infoGain=1e-9
    Left=[]
    Right=[]
    Threshold=None
    Feature=None
    
        
    for feature in range(RowSize-1):
        allValsinFeature=[Data[x][feature] for x in range(len(Data))]
        minVal=min(allValsinFeature)
        maxVal=max(allValsinFeature)
        Tvals = np.linspace(minVal,maxVal,16,endpoint=False)[1:]
        for threshold in Tvals:
            left=[]
            right=[]
            for inputs in Data:
                if inputs[feature]<threshold:
                    left.append([inputs[x] for x in [*range(0,feature),*range(feature+1,RowSize)]])
                else:
                    right.append([inputs[x] for x in [*range(0,feature),*range(feature+1,RowSize)]])
            if(len(left)>0 and len(right)>0):
                delGini=gini(Data)-len(left)/len(Data)*gini(left)-len(right)/len(Data)*gini(right)
                if delGini > infoGain:
                    infoGain=delGini
                    Left=left
                    Right=right
                    Threshold=threshold
                    Feature=feature
    return {"threshold":Threshold,"feature":Feature,"Left":Left,"Right":Right}


class DecisionTree():
    tree={}
    #learn function
    def learn(self,Data):
        tree={'feature':None, 'Value':None ,'threshold':None,'leftTree':{},'rightTree':{}}
        # if len(Data)==0 then the node is essentially not splitting
        if len(Data)>0:
            # there should be at least 1 feature to split on and all the data shouldn't go to a single node
            if len(Data[0])>=2 and gini(Data)!=0:
                best_split=BestSplit(Data)
                tree['feature']=best_split["feature"]
                tree['threshold']=best_split["threshold"]
                leftTree={}
                rightTree={}
                leftTree=DecisionTree.learn(self,best_split["Left"])
                rightTree=DecisionTree.learn(self,best_split["Right"])
                tree['leftTree']=leftTree
                tree['rightTree']=rightTree
                return tree 
            else :
                tree['leftTree']= None
                tree['rightTree']= None 
                rowsize=len(Data[0])
                labels=[Data[x][rowsize-1] for x in range(len(Data))]
                tree['Value']=max(labels,key=labels.count)
                return tree    

    def classify(self,tree,test_instance):
        if tree:
            if(tree['Value']) is not None:
                return tree['Value'] 
            else:
                f = test_instance[tree['feature']]
                t=[test_instance[x] for x in [*range(0,tree['feature']),*range(tree['feature']+1,len(test_instance))]] 
                if(f<tree['threshold']):
                    return self.classify(tree['leftTree'],t)
                else:
                    return self.classify(tree['rightTree'],t)
    

def run_decision_tree():
    BestTree={}
    BestAccuracy=0
    avgAccuracy=0
    # Load data set
    # random.shuffle(Tdata)

    # Split training/test sets
    K = 10
    for kfold in range(K):
        training_set = [x for i, x in enumerate(Tdata) if i % K != kfold]
        test_set = [x for i, x in enumerate(Tdata) if i % K == kfold]
        Tree = DecisionTree()
        # Construct a tree using training set
        tree=Tree.learn( training_set )
        
        # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = Tree.classify(tree,instance[:-1] )
            results.append( result == instance[-1])

        # Accuracy
        accuracy = float(results.count(True))/float(len(results))
        if BestAccuracy<accuracy:
            BestAccuracy=accuracy
            BestTree=tree
        avgAccuracy=avgAccuracy+accuracy
        print("kfold: %d, accuracy: %.4f" % (kfold+1, accuracy))
        

        # Writing results to a file (DO NOT CHANGE)
        f = open("Chirag-Mehta-"+"result.txt", "w")
        f.write("accuracy: %.4f" % accuracy)
        f.close()
    avgAccuracy=avgAccuracy/K
    print("Best accuracy: %f, average accuracy: %f" %(BestAccuracy,avgAccuracy))
    f = open("Chirag-Mehta-"+"result.txt", "w")
    f.write("average accuracy: %.4f" % avgAccuracy)
    f.close()
if __name__ == "__main__":
    run_decision_tree()

kfold: 1, accuracy: 0.8510
kfold: 2, accuracy: 0.8347
kfold: 3, accuracy: 0.8082
kfold: 4, accuracy: 0.8224
kfold: 5, accuracy: 0.8347
kfold: 6, accuracy: 0.7959
kfold: 7, accuracy: 0.8184
kfold: 8, accuracy: 0.7898
kfold: 9, accuracy: 0.7996
kfold: 10, accuracy: 0.7873
Best accuracy: 0.851020, average accuracy: 0.814201


In [19]:
# Improvement strategy used: pre-prunning

import random
import numpy as np
import csv
with open("wine-dataset.csv") as f:
    next(f, None)
    Tdata=[]
    for line in csv.reader(f,delimiter=","):
        row=[float(x) for x in line]
        Tdata.append(row)

# entropy function
def entropy(data):
    
    lastIndex=len(data[0])-1
    n0,n1=0,0
    for i in data:
        if i[lastIndex]==0:
            n0=n0+1
        else:
            n1=n1+1
    p1=n0/(n1+n0)
    p2=1-p1
    # prunning
    if max(p1,p2) > 0.95:
        return 0
    if p1 != 0:
        p1=-p1*np.log(p1)
    if p2 != 0:
        p2=-p2*np.log(p2)
    return p1+p2


# The Best split function
def BestSplit(Data):
    RowSize=len(Data[0])
    if RowSize<=1:
        exit
    infoGain=1e-9
    Left=[]
    Right=[]
    Threshold=None
    Feature=None
    
        
    for feature in range(RowSize-1):
        allValsinFeature=[Data[x][feature] for x in range(len(Data))]
        minVal=min(allValsinFeature)
        maxVal=max(allValsinFeature)
        Tvals = np.linspace(minVal,maxVal,16,endpoint=False)[1:]
        for threshold in Tvals:
            left=[]
            right=[]
            for inputs in Data:
                if inputs[feature]<threshold:
                    left.append([inputs[x] for x in [*range(0,feature),*range(feature+1,RowSize)]])
                else:
                    right.append([inputs[x] for x in [*range(0,feature),*range(feature+1,RowSize)]])
            if(len(left)>0 and len(right)>0):
                delEntropy=entropy(Data)-len(left)/len(Data)*entropy(left)-len(right)/len(Data)*entropy(right)
                if delEntropy > infoGain:
                    infoGain=delEntropy
                    Left=left
                    Right=right
                    Threshold=threshold
                    Feature=feature
    return {"threshold":Threshold,"feature":Feature,"Left":Left,"Right":Right}


class DecisionTree():
    tree={}

    def learn(self,Data):
        tree={'feature':None, 'Value':None ,'threshold':None,'leftTree':{},'rightTree':{}}
        # if len(Data)==0 then the node is essentially not splitting
        if len(Data)>0:
            # there should be at least 1 feature to split on and all the data shouldn't go to a single node
            if len(Data[0])>=2 and entropy(Data)!=0:
                best_split=BestSplit(Data)
                tree['feature']=best_split["feature"]
                tree['threshold']=best_split["threshold"]
                leftTree={}
                rightTree={}
                leftTree=DecisionTree.learn(self,best_split["Left"])
                rightTree=DecisionTree.learn(self,best_split["Right"])
                tree['leftTree']=leftTree
                tree['rightTree']=rightTree
                return tree 
            else :
                tree['leftTree']= None
                tree['rightTree']= None 
                rowsize=len(Data[0])
                labels=[Data[x][rowsize-1] for x in range(len(Data))]
                tree['Value']=max(labels,key=labels.count)
                return tree    

    def classify(self,tree,test_instance):
        if tree:
            if(tree['Value']) is not None:
                return tree['Value'] 
            else:
                f = test_instance[tree['feature']]
                t=[test_instance[x] for x in [*range(0,tree['feature']),*range(tree['feature']+1,len(test_instance))]] 
                if(f<tree['threshold']):
                    return self.classify(tree['leftTree'],t)
                else:
                    return self.classify(tree['rightTree'],t)
    

def run_decision_tree():
    BestTree={}
    BestAccuracy=0
    avgAccuracy=0

    # random.shuffle(Tdata)

    # Split training/test sets
    K = 10
    for kfold in range(K):
        training_set = [x for i, x in enumerate(Tdata) if i % K != kfold]
        test_set = [x for i, x in enumerate(Tdata) if i % K == kfold]
        Tree = DecisionTree()
        tree=Tree.learn( training_set )
        
        # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = Tree.classify(tree,instance[:-1] )
            results.append( result == instance[-1])

        # Accuracy
        accuracy = float(results.count(True))/float(len(results))
        if BestAccuracy<accuracy:
            BestAccuracy=accuracy
            BestTree=tree
        avgAccuracy=avgAccuracy+accuracy
        print("kfold: %d, accuracy: %.4f" % (kfold+1, accuracy))
        

        # Writing results to a file (DO NOT CHANGE)
        f = open("Chirag-Mehta-"+"result.txt", "w")
        f.write("accuracy: %.4f" % accuracy)
        f.close()
    avgAccuracy=avgAccuracy/K
    print("Best accuracy: %f, average accuracy: %f" %(BestAccuracy,avgAccuracy))
    f = open("Chirag-Mehta-"+"result.txt", "w")
    f.write("average accuracy: %.4f" % avgAccuracy)
    f.close()
if __name__ == "__main__":
    run_decision_tree()

kfold: 1, accuracy: 0.8265
kfold: 2, accuracy: 0.8327
kfold: 3, accuracy: 0.8000
kfold: 4, accuracy: 0.8184
kfold: 5, accuracy: 0.8245
kfold: 6, accuracy: 0.8122
kfold: 7, accuracy: 0.8204
kfold: 8, accuracy: 0.7898
kfold: 9, accuracy: 0.8078
kfold: 10, accuracy: 0.8323
Best accuracy: 0.832653, average accuracy: 0.816457
